# UAVSAR Interferogram Processing

**Author**: Forrest Williams - Jet Propulsion Laboratory

This notebook provides a working example of end-to-end processing of UAVSAR interferograms through the ISCE application **stripmapApp.py**.

## Prep A: Initial setup of the notebook
Below we set up the directory structure for this notebook exercise. In addition, we load the required modules into our python environment using the **`import`** command.

In [ ]:
import glob
import os
import stat
import sys
from datetime import datetime
import itertools

## Specify DEM path
dem_path = '/u/sar-r0/fielding/Calif/SF_Bay/topo/USGS/1_3asec'

## Defining the home and data directories at the processing location
tutorial_home_dir = os.path.abspath(os.getcwd())
work_dir = os.path.abspath(os.getcwd())
## Manually change work_dir if preferred. E.g.:
#work_dir = '/u/sar-r2/fwilliam/Calif/UAVSAR/Stacks/Haywrd_05502_01-S2'
## Manually specify remote login. E.g.:
#user_remote=fwilliam@137.78.249.109
print("Tutorial directory: ", tutorial_home_dir)
print("Work directory: ", work_dir)

## Enter working directory
os.chdir(work_dir)

## Prep B: Download data
First attempt to download data directly with the bash script.

In [ ]:
!chmod u+x Haywrd_0550201.sh
!./Haywrd_0550201.sh

If downloading fails with a `connection denied` error, it is an access issue associated with your server's firewall. For this case, modify `work_dir` in **Prep A** above to reflect the desired path you wish to push data to on your remote server. Then re-execute this cell on your local machine, and finally continue working through the rest of the notebook on your remote server.

In [14]:
# Create the batch script for downloading the data
with open('Haywrd_0550201.sh','r') as f:
    lines = f.readlines()

files = [x.split(' ')[1][:-1] for x in lines]
names = [x.split('/')[-1] for x in files]

# Check if any file failed to download
downloaded_file = [os.path.exists(i) for i in names]
if False in downloaded_file:
    batch_lines = ['@ECHO OFF\n'] + [f'CALL CURL {x} -o {y}\n' for x,y in zip(files,names)]

    with open('..\\Haywrd_0550201.bat', 'w+') as f:
        f.writelines(batch_lines)

    # Create the batch script for transferring the data
    complete = glob.glob('..\Hayw*')
    remote_path = '{}:{}'.format(user_remote,work_dir)
    transfer_lines = ['@ECHO OFF\n'] + [f'CALL scp {os.path.basename(x)} {remote_path}\n' for x in complete]
    with open('..\\transfer.bat', 'w+') as f:
        f.writelines(transfer_lines)

## Perform ISCE Processing

### Create Run Files

In [ ]:
!ls

In [3]:
!mkdir downloadHH

In [4]:
!mv Haywar* downloadHH/

In [5]:
!mv downloadHH/*.dop .

In [ ]:
# !prepareUAVSAR_coregStack.py -h
!prepareUAVSAR_coregStack.py -i downloadHH -o SLC -d Haywar_05502_01_BC.dop -s 2

Some directory setup needs to be done before we can run `stackStripMap.py`:

In [ ]:
!ls

In [13]:
!mkdir merged

In [ ]:
!cd merged

In [15]:
!ln -s ../SLC

In [ ]:
!cd ..

In [21]:
## Link DEM to working directory
os.symlink(dem_path, 'DEM')

In [ ]:
!stackStripMap.py -s SLC/ -d ./1_3asec/BayArea_USGS_13.dem.wgs84 -a 8 -r 2 -W interferogram --nofocus -S uavsar_stack -t 600 --filter_strength 0.2

It looks like 600 is not a large enough time delta to create a connected network, lets determine what the minimum time delta needed is:

In [ ]:
d = ['20090220', '20091119', '20100115', '20100301', '20100423', '20101110', '20110712', '20111103', '20120413', '20121102', '20140528', '20141112', '20151008', '20160324', '20170403', '20171020', '20180801', '20200910', '20201007', '20210601']
dates = [datetime.strptime(x,'%Y%m%d') for x in d]
dates.sort()
difference = [x-y for x,y in zip(dates,[dates[0]]+dates[0:-1])]
max(difference)

In [ ]:
!stackStripMap.py -s SLC/ -d ./DEM/BayArea_USGS_13.dem.wgs84 -a 8 -r 2 -W interferogram --nofocus -S uavsar_stack -t 800 --filter_strength 0.2

In [ ]:
!ls run_files

In [ ]:
!run.py -h

In [ ]:
!nproc

### Excecute Run Files

For long-running scripts, it's better to run the commands within a shell with nohup specified instead of from inside the notebook because jupyter lab does not handle ssh disconnections well. Also, for **UAVSAR data only steps 1 and 8 need to be run** (coregister DEM and create interferograms). The cell below generates two .tsch that run each respectively and log the results. Copy the lines printed by this cell, then run them in the shell.

In [ ]:
wd = os.getcwd()

header = """#! /bin/tcsh

date
#Start time
#begin=`date +"%s"`

# RUN
"""

master = f'nohup source {os.path.join(wd,"run_files","run_01_reference")} > master.log'
igrams = f'nohup source {os.path.join(wd,"run_files","run_08_igram")} > igrams.log'

footer = """

# Print time after running
#echo Walltime : $(expr `date +"%s"` - $begin)          # Seconds
date
"""

cmds = {'master':master,'igrams':igrams}
for key in cmds:
    name = f'run-{key}.tcsh' 
    with open(name, 'w') as f:
        f.write(header)
        f.write(cmds[key])
        f.write(footer)
    st = os.stat(name)
    os.chmod(name, st.st_mode | stat.S_IEXEC)
    print(f'./{name} ; python JPL_internship/emailme.py')

### Step 1 Log
Took about 30 minutes to run

In [ ]:
!cat master.log

### Step 8 Log
This step took 57 hours to complete (~1 hour per interferogram)

In [ ]:
n_pairs = 56
with open('igrams.log','r') as f:
    lines = f.readlines()

n_completions = len([x for x in lines if '*** Normal Completion  ***' in x])

if n_pairs == n_completions:
    print('All interferograms completed successfully!')
else:
    
    print(f'There was a problem with the run :( {n_completions} interferograms done.')

In [ ]:
!cat igrams.log